In [27]:
import xml.etree.ElementTree as ET
import csv
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import numpy as np
from glob import glob
import h5py
import pandas as pd

xml_path = '..//..//datasets//image//INbreast//INbreast Release 1.0//AllXML//20586908.xml'
# C:\Users\User\Desktop\ML\project\ML_Team28\datasets\image\INbreast\INbreast Release 1.0\AllXML\20586908.xml
tree = ET.parse(xml_path)
root = tree.getroot()
def print_xml_data(element, depth=0):
    # 打印当前元素的标签和文本内容
    print('  ' * (depth-1) + f"<{element.tag}>")
    print('  ' * depth + f"{element.text}")

    # 递归遍历子元素
    for child in element:
        print_xml_data(child, depth + 1)
# print_xml_data(root)

def parse_xml_to_dict(element):
    # 递归函数，将 XML 元素解析为字典
    result = {}
    for child in element:
        if len(child) == 0:
            result[child.tag] = child.text
        else:
            result[child.tag] = parse_xml_to_dict(child)
    return result

# 解析 XML 文件
tree = ET.parse(xml_path)
root = tree.getroot()

# 将 XML 数据解析为字典列表
data = []
for element in root:
    print(element)
    data.append(parse_xml_to_dict(element))
print(data)
# 转换为 DataFrame
df = pd.DataFrame(data)
# print(df)
# df.to_csv('..//..//datasets//image//INbreast//INbreast Release 1.0//20586908.csv')

<Element 'dict' at 0x000001CF361583B0>
[{'key': 'Images', 'array': {'dict': {'key': 'ROIs', 'integer': '42', 'array': {'dict': {'key': 'Type', 'real': '1514', 'string': 'Calcification', 'integer': '19', 'array': {'string': '(3044.800049, 3259.790039)'}}}}}}]


In [53]:
from skimage.draw import polygon
import numpy as np
import plistlib

def load_inbreast_mask(mask_path, imshape=(4084, 3328)):
    """
    This function loads a osirix xml region as a binary numpy array for INBREAST
    dataset
    @mask_path : Path to the xml file
    @imshape : The shape of the image as an array e.g. [4084, 3328]
    return: numpy array where positions in the roi are assigned a value of 1.
    """

    def load_point(point_string):
        x, y = tuple([float(num) for num in point_string.strip('()').split(',')])
        return y, x

    mask = np.zeros(imshape)
    with open(mask_path, 'rb') as mask_file:
        plist_dict = plistlib.load(mask_file, fmt=plistlib.FMT_XML)['Images'][0]
        plist_df = pd.DataFrame(plist_dict)
        # plist_df.to_csv('..//..//datasets//image//INbreast//INbreast Release 1.0//20586908.csv')
        numRois = plist_dict['NumberOfROIs']
        rois = plist_dict['ROIs']
        assert len(rois) == numRois
        for roi in rois:
            numPoints = roi['NumberOfPoints']
            points = roi['Point_px']
            assert numPoints == len(points)
            points = [load_point(point) for point in points]
            if len(points) <= 2:
                for point in points:
                    mask[int(point[0]), int(point[1])] = 1
            else:
                x, y = zip(*points)
                x, y = np.array(x), np.array(y)
                poly_x, poly_y = polygon(x, y, shape=imshape)
                mask[poly_x, poly_y] = 1
    return mask
x = load_inbreast_mask(xml_path)
# i=0
# for n in x:
#     i+=1
#     if n.any() == 1:
#         print(n,i)
# print(load_inbreast_mask(xml_path))

[0. 0. 0. ... 0. 0. 0.] 466
[0. 0. 0. ... 0. 0. 0.] 467
[0. 0. 0. ... 0. 0. 0.] 669
[0. 0. 0. ... 0. 0. 0.] 678
[0. 0. 0. ... 0. 0. 0.] 721
[0. 0. 0. ... 0. 0. 0.] 818
[0. 0. 0. ... 0. 0. 0.] 819
[0. 0. 0. ... 0. 0. 0.] 820
[0. 0. 0. ... 0. 0. 0.] 821
[0. 0. 0. ... 0. 0. 0.] 822
[0. 0. 0. ... 0. 0. 0.] 823
[0. 0. 0. ... 0. 0. 0.] 824
[0. 0. 0. ... 0. 0. 0.] 825
[0. 0. 0. ... 0. 0. 0.] 826
[0. 0. 0. ... 0. 0. 0.] 827
[0. 0. 0. ... 0. 0. 0.] 828
[0. 0. 0. ... 0. 0. 0.] 838
[0. 0. 0. ... 0. 0. 0.] 844
[0. 0. 0. ... 0. 0. 0.] 873
[0. 0. 0. ... 0. 0. 0.] 984
[0. 0. 0. ... 0. 0. 0.] 985
[0. 0. 0. ... 0. 0. 0.] 986
[0. 0. 0. ... 0. 0. 0.] 987
[0. 0. 0. ... 0. 0. 0.] 988
[0. 0. 0. ... 0. 0. 0.] 989
[0. 0. 0. ... 0. 0. 0.] 990
[0. 0. 0. ... 0. 0. 0.] 991
[0. 0. 0. ... 0. 0. 0.] 992
[0. 0. 0. ... 0. 0. 0.] 993
[0. 0. 0. ... 0. 0. 0.] 994
[0. 0. 0. ... 0. 0. 0.] 995
[0. 0. 0. ... 0. 0. 0.] 996
[0. 0. 0. ... 0. 0. 0.] 997
[0. 0. 0. ... 0. 0. 0.] 998
[0. 0. 0. ... 0. 0. 0.] 999
[0. 0. 0. ... 0. 0. 

In [69]:
from read_roi import read_roi_file
# import ImagejRoi
# import load_imagej from 'read_roi.py'
# from r_roi import load_imagej
# from r_roi2 import read_roi

import numpy as np

def read_roi(fileobj):
    '''
    points = read_roi(fileobj)
    Read ImageJ's ROI format
    '''
# This is based on:
# http://rsbweb.nih.gov/ij/developer/source/ij/io/RoiDecoder.java.html
# http://rsbweb.nih.gov/ij/developer/source/ij/io/RoiEncoder.java.html


    SPLINE_FIT = 1
    DOUBLE_HEADED = 2
    OUTLINE = 4
    OVERLAY_LABELS = 8
    OVERLAY_NAMES = 16
    OVERLAY_BACKGROUNDS = 32
    OVERLAY_BOLD = 64
    SUB_PIXEL_RESOLUTION = 128
    DRAW_OFFSET = 256


    pos = [4]
    def get8():
        pos[0] += 1
        s = fileobj.read(1)
        if not s:
            raise IOError('readroi: Unexpected EOF')
        return ord(s)

    def get16():
        b0 = get8()
        b1 = get8()
        return (b0 << 8) | b1

    def get32():
        s0 = get16()
        s1 = get16()
        return (s0 << 16) | s1

    def getfloat():
        v = np.int32(get32())
        return v.view(np.float32)

    magic = fileobj.read(4)
    if magic != 'Iout':
        raise IOError('Magic number not found')
    version = get16()

    # It seems that the roi type field occupies 2 Bytes, but only one is used
    roi_type = get8()
    # Discard second Byte:
    get8()

    if not (0 <= roi_type < 11):
        raise ValueError('roireader: ROI type %s not supported' % roi_type)

    if roi_type != 7:
        raise ValueError('roireader: ROI type %s not supported (!= 7)' % roi_type)

    top = get16()
    left = get16()
    bottom = get16()
    right = get16()
    n_coordinates = get16()

    x1 = getfloat() 
    y1 = getfloat() 
    x2 = getfloat() 
    y2 = getfloat()
    stroke_width = get16()
    shape_roi_size = get32()
    stroke_color = get32()
    fill_color = get32()
    subtype = get16()
    if subtype != 0:
        raise ValueError('roireader: ROI subtype %s not supported (!= 0)' % subtype)
    options = get16()
    arrow_style = get8()
    arrow_head_size = get8()
    rect_arc_size = get16()
    position = get32()
    header2offset = get32()

    if options & SUB_PIXEL_RESOLUTION:
        getc = getfloat
        points = np.empty((n_coordinates, 2), dtype=np.float32)
    else:
        getc = get16
        points = np.empty((n_coordinates, 2), dtype=np.int16)
    points[:,1] = [getc() for i in xrange(n_coordinates)]
    points[:,0] = [getc() for i in xrange(n_coordinates)]
    points[:,1] += left
    points[:,0] += top
    points -= 1
    return points

def read_roi_zip(fname):
    import zipfile
    with zipfile.ZipFile(fname) as zf:
        return [read_roi(zf.open(n))
                    for n in zf.namelist()]

roi_file_path = '..//..//datasets//image//INbreast//INbreast Release 1.0//AllROI//20586908.roi'
img = read_roi_file(roi_file_path)
print(img)
read_roi(img)
# roi2 = ImagejRoi.fromfile(roi_file_path)
# read_roi_file(roi_file_path)
# print(roi2)

UnboundLocalError: local variable 'channel' referenced before assignment